# Installing oauth2-proxy

Oauth2-proxy provides OAuth 2.0 authentication as a proxy for other applications. 

## Prerequisites
- [Learning Jupyter](../getting-started/learning_jupyter.ipynb)
- Install Jupyter bash kernel:
  - Windows... coming soon!
  - [Mac](../getting-started/mac/install_jupyter_bash_kernel.ipynb)
  - [Linux](../getting-started/linux/install_jupyter_bash_kernel.ipynb)
- [Install Kubernetes](../kubernetes/install_k8s_cluster.ipynb)
- Configure DNS
  - Mac: [Install DNSmasq](../dnsmasq/mac/install_dnsmasq.ipynb)
  - Linux: [Install DNSmasq](../../dnsmasq/linux/install_dnsmasq.ipynb)
  - Windows: Install Acrylic... coming soon!
- [Install Monitoring](../monitoring/install_kube-prometheus-stack.ipynb)
- [Install Logging](../logging/install_loki.ipynb)
- [Install cert-manager](../cert-manager/install_cert-manager.ipynb)
- [Install Ingress Controller](../ingress/install_ingress-nginx.ipynb)
- [Install Postgres](../postgres/install_postgres.ipynb)
- [Install OpenLDAP](../openldap/install_openldap.ipynb)
- [Install Keycloak](../keycloak/install_keycloak.ipynb)

### Add the oauth2-proxy Helm repository

We begin by adding the helm repository that contains the oauth2-proxy helm chart.  We'll use this chart below to install oauth2-proxy.

In [1]:
helm repo add oauth2-proxy https://oauth2-proxy.github.io/manifests

"oauth2-proxy" already exists with the same configuration, skipping


### Install oauth2-proxy

Next we install the oauth2-proxy application using helm.  We'll use a helm values file that is configured to the Keycload realm `ldap-realm`.  It uses the Keycloak client `oauth2-proxy` by configurig the clientID and clientSecret with values used in Keycloak.  It exposes the login & token refresh OAuth 2.0 endpoints using an Ingress object at the URL, `oauth2.k8s.nephtek.com`.  We'll use this endpoint as the login redirect URL for Prometheus and Alertmanager by adding annotations to the Ingress objects for Prometheus & Alertmanager.  

The ingress object for the proxy also could be set to a subpath of an existing URL.  With this approach, the oauth2-proxy would have an ingress object for each application that it provided access to, with the URL set to a subpath of the application's URL instead of at `oauth2.k8s.nephtek.com`.

In [2]:
helm upgrade --install oauth2-proxy oauth2-proxy/oauth2-proxy -n=oauth2-proxy --create-namespace -f values.rancher-desktop.yaml

Release "oauth2-proxy" does not exist. Installing it now.
NAME: oauth2-proxy
LAST DEPLOYED: Sun Oct  2 14:08:29 2022
NAMESPACE: oauth2-proxy
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
To verify that oauth2-proxy has started, run:

  kubectl --namespace=oauth2-proxy get pods -l "app=oauth2-proxy"


### Modify Ingress objects to use oauth2-proxy

We'll use kustomize to modify the ingress objects for Prometheus and Alertmanager to make use of oauth2-proxy. 

In [3]:
kubectl apply -k kustomize/overlays/rancher-desktop

ingress.networking.k8s.io/alertmanager-oauth2 created
ingress.networking.k8s.io/prometheus-oauth2 created


### Finished
We've transparently added authentication to Prometheus and Alertmanager, neither of which support authentication natively, all by adding simple annotations to their Ingress objects.

### Next Steps
- [Install kube-oidc-proxy](../kube-oidc-proxy/install_kube-oidc-proxy.ipynb)